# Step 33, 뉴턴 방법으로 푸는 최적화(자동 계산)

2차 미분을 자동으로 계산해본다.  
미분값이 제대로 구했음을 확인한다.  
뉴턴 방법을 사용해 최적화를 한다.

## 33.1 2차 미분 계산하기 

$y=x^4-2x^2$를 2차 미분을 해본다.

그전에 core.py 코드를 수정해줘야한다.
~~~python
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        # x = self.inputs[0].data
        x, = self.inputs
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)
~~~

In [1]:
if '__file__' in globals():
    import os, sys 
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np 
from dezero import Variable 

def f(x):
    y = x**4 - 2*x**2 
    return y 

x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)   # 1
print(x.grad)

# 두번째 역전파 진행 
gx = x.grad                     # 2
gx.backward()                   # 3
print(x.grad)

variable(24.0)
variable(68.0)


1. y.backward(create_graph=True)에 의해 첫번째 역전파가 진행된다.  
이때 인수 create_graph를 True로 지정하여 역전파 계산에 대해서도 계산 그래프를 만들게 한다.  

2. 역전파 계산 그래프에 다시 역전파를 한다. x의 2차 미분을 계산해야 하므로  
gx = x.grad 코드로 y의 x에 대한 미분값을 꺼낸다.  

3. 미분값인 gx에서 한번더 역전파를 한다. gx의 x에 대한 미분을 구한다. : 2차 미분 

$y'' = 12x^2 - 4$, $x=2$일때 44이다. 결과가 다른 이유는  

1차 미분 결과인 24에 2차 미분결과인 44가 더해진 값이 나오기 때문이다.  
즉, Variable에 미분값이 남아 있는 상태에서 새로운 역전파를 수행했디 때문에 새로운 미분값이 '더해진' 것이다.

해결 : 새로운 계산을 하기전에 Variable의 미분값을 '재설정'해야 한다.

In [2]:
x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)   # 1
print(x.grad)

# 두번째 역전파 진행 
gx = x.grad                     # 2
x.cleargrad()   # gx.backward()를 하기전에 x.cleargrad()를 추가한다. 
gx.backward()                   # 3
print(x.grad)

variable(24.0)
variable(44.0)


## 33.2 뉴턴 방법을 활용한 최적화 

뉴턴 방법을 활용한 최적화 수식 

$$x \leftarrow x - \frac{f'(x)}{f''(x)}$$

In [3]:
import numpy as np 
from dezero import Variable

def f(x):
    y = x**4 - 2*x**2 
    return y 

x = Variable(np.array(2.0))
iters = 10 

for i in range(iters):
    print(i, x)

    y = f(x)
    x.cleargrad()
    y.backward(create_graph=True)

    gx = x.grad  # 미분 1번
    x.cleargrad()
    gx.backward()
    gx2 = x.grad # 미분 2번

    x.data -= gx.data / gx2.data 
    # 최솟값을 갖는 x의 위치는 1,-1이다.

0 variable(2.0)
1 variable(1.4545454545454546)
2 variable(1.1510467893775467)
3 variable(1.0253259289766978)
4 variable(1.0009084519430513)
5 variable(1.0000012353089454)
6 variable(1.000000000002289)
7 variable(1.0)
8 variable(1.0)
9 variable(1.0)
